In [1]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu'
os.environ['PATH'] = '/home/fandav/.local/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/anaconda3/envs/jupyter/bin:/opt/anaconda3/bin/:/snap/bin'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ

environ{'USER': 'fandav',
        'JUPYTERHUB_HOST': '',
        'LD_LIBRARY_PATH': '/usr/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu',
        'JUPYTERHUB_USER': 'fandav',
        'SHLVL': '0',
        'HOME': '/home/fandav',
        'JUPYTERHUB_API_URL': 'http://127.0.0.1:8081/hub/api',
        'JUPYTERHUB_OAUTH_CALLBACK_URL': '/user/fandav/oauth_callback',
        'SLURM': '1',
        'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-fandav',
        'PATH': '/home/fandav/.local/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/anaconda3/envs/jupyter/bin:/opt/anaconda3/bin/:/snap/bin',
        'JUPYTERHUB_ACTIVITY_URL': 'http://127.0.0.1:8081/hub/api/users/fandav/activity',
        'LANG': 'en_US.UTF-8',
        'SHELL': '/bin/zsh',
        'JUPYTERHUB_SERVICE_PREFIX': '/user/fandav/',
        'PWD': '/home/fandav',
        'JUPYTERHUB_API_TOKEN': '115cb36d01164469b7b013199ce67d48',
        'JUPYTERHUB_BASE_URL': '/',
        'XDG_DATA_DIRS': '/usr/local/share:/usr/share:/v

In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pandas as pd

# To find out which devices your operations and tensors are assigned to
tf.debugging.set_log_device_placement(True)

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [4]:
'''
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
'''

'\nfrom tensorflow.compat.v1 import ConfigProto\nfrom tensorflow.compat.v1 import InteractiveSession\n\nconfig = ConfigProto()\nconfig.gpu_options.allow_growth = True\nsession = InteractiveSession(config=config)\n'

In [5]:
physical_cpus = tf.config.list_physical_devices('CPU') 
logical_cpus = tf.config.list_logical_devices('GPU')
print(len(physical_cpus), "Physical CPUs,", len(logical_cpus), "Logical CPUs")

1 Physical CPUs, 0 Logical CPUs


In [6]:
'''
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
'''

'\ngpus = tf.config.list_physical_devices(\'GPU\')\nif gpus:\n    try:\n        for gpu in gpus:\n            tf.config.experimental.set_memory_growth(gpu, True)\n            logical_gpus = tf.config.list_logical_devices(\'GPU\')\n            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")\n    except RuntimeError as e:\n        print(e)\n'

In [7]:
# tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# model
model = TFBertModel.from_pretrained('bert-base-uncased')
model.summary()

Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TruncatedNormal in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device

In [8]:
dataset = pd.read_csv('dataset.csv', index_col=0)
#display(dataset.loc['Content'].values)

In [9]:
dataset_content = dataset.loc['Content'].fillna('').values
#print(dataset_content)

In [10]:
encoded_strings = [tokenizer.encode(content, max_length=512, pad_to_max_length=True) for content in dataset_content]

In [11]:
encoded_vectors = tf.constant(encoded_strings)

outputs = model(encoded_vectors)
last_hidden_states = outputs[0]

In [12]:
print(np.shape(outputs))
print(np.shape(outputs[0]))
print(np.shape(outputs[1]))

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:CPU:0
(2, 50)
(50, 512, 768)
(50, 768)


In [13]:
_, num_articles = np.shape(outputs)
article_names = [f'Article {idx}' for idx in range(num_articles)]

In [14]:
article_embeddings = outputs[1]

# Look at some dimensionality reduction plots

In [15]:
import matplotlib.pyplot as plt
%matplotlib widget
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd

In [16]:
# vector embeddings of each token
vec_embeddings = article_embeddings.numpy()

# color for each token for visuals
token_color = [np.random.rand(3,) for _ in range(num_articles)]

In [17]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(vec_embeddings)
print(np.shape(pca_result))

(50, 2)


In [18]:
df = pd.DataFrame(pca_result, columns=['pca_0', 'pca_1'], index=article_names)
display(df)

,pca_0,pca_1
Article 0,2.476506,-0.836434
Article 1,-6.809633,4.225299
Article 2,-0.342575,-0.542970
Article 3,2.047678,0.239901
Article 4,1.006995,-0.429198
Article 5,1.516522,-1.332965
Article 6,-6.809633,4.225299
Article 7,-0.707807,-1.102196
Article 8,6.287801,4.657908
Article 9,-1.052074,-1.013302


In [19]:
plt.figure(figsize=(16,8))
sns.scatterplot(
    x='pca_0', y='pca_1',
    data=df,
    legend='full'
)
plt.grid()

ax = plt.gca()
for (k, v), color in zip(df.iterrows(), token_color):
    ax.annotate(k, v, c=color)

#for idx, (article, color) in enumerate(zip(article_names, token_color)):
#    plt.text(-7 + idx*1, -10, article, fontsize='medium', color=color)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
tsne = TSNE(n_components=2, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(vec_embeddings)
print(np.shape(tsne_results))

(50, 2)


In [21]:
df = pd.DataFrame(tsne_results, columns=['tsne_0', 'tsne_1'], index=article_names)
display(df)

,tsne_0,tsne_1
Article 0,35.407242,-34.709393
Article 1,-34.008278,67.447006
Article 2,53.292747,23.318266
Article 3,111.164200,5.378065
Article 4,20.139839,28.677109
Article 5,-6.598174,29.036839
Article 6,50.095341,-58.057308
Article 7,-31.634842,-37.745350
Article 8,59.726364,52.087086
Article 9,0.881645,-37.546097


In [22]:
plt.figure(figsize=(16,8))
sns.scatterplot(
    x="tsne_0", y="tsne_1",
    data=df,
    legend="full"
)
plt.grid()

ax = plt.gca()
for (k, v), color in zip(df.iterrows(), token_color):
    ax.annotate(k, v, c=color)

#for idx, (token, color) in enumerate(zip(tokens, token_color)):
#    plt.text(-400 + idx*50, -400, token, fontsize='medium', color=color)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Dimensionality Reduction projection into 3D space

In [23]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [24]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(vec_embeddings)
print(np.shape(pca_result))

(50, 3)


In [25]:
df = pd.DataFrame(pca_result, columns=['pca_0', 'pca_1', 'pca_2'], index=article_names)
display(df)

,pca_0,pca_1,pca_2
Article 0,2.476508,-0.836434,-0.553262
Article 1,-6.809634,4.225299,-0.437597
Article 2,-0.342575,-0.542971,0.360609
Article 3,2.047678,0.239902,-0.044982
Article 4,1.006996,-0.429198,0.066745
Article 5,1.516523,-1.332965,-0.525635
Article 6,-6.809634,4.225300,-0.437597
Article 7,-0.707807,-1.102197,0.265242
Article 8,6.287798,4.657910,0.385951
Article 9,-1.052074,-1.013303,0.349429


In [26]:
fig = plt.figure()
ax = Axes3D(fig)

ax.scatter(df['pca_0'], df['pca_1'], df['pca_2'], c=token_color)
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
print(np.shape(vec_embeddings))

(50, 768)


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Conv1D, MaxPooling1D, InputLayer, Input

In [51]:
'''
model = Sequential([
    InputLayer(input_shape=(768)),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1),
])
'''

model = Sequential([
    InputLayer(input_shape=(768,1)),
    Conv1D(16, 3, padding='valid', activation='relu', input_shape=(768, 1)),
    MaxPooling1D(),
    Conv1D(32, 5, padding='valid', activation='relu', input_shape=(383, 16)),
    MaxPooling1D(),
    Conv1D(64, 3, padding='valid', activation='relu', input_shape=(383, 16)),
    MaxPooling1D(),
    Conv1D(128, 3, padding='valid', activation='relu', input_shape=(383, 16)),
    MaxPooling1D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])
    
    
'''
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, name='predictions')(x)
'''
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 766, 16)           64        
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 383, 16)           0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 379, 32)           2592      
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 189, 32)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 187, 64)           6208      
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 93, 64)            0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 91, 128)         

In [55]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [31]:
np.shape(vec_embeddings[0:10].T)

(768, 10)

In [32]:
print(vec_embeddings[0:10])

[[-0.5096348  -0.6940365  -0.9991142  ... -0.9876248  -0.5398137
  -0.00246001]
 [-0.16403927 -0.26082677 -0.89390785 ... -0.6002618  -0.24370384
   0.06840346]
 [-0.3667999  -0.58417225 -0.9970588  ... -0.9668266  -0.41875827
   0.03159428]
 ...
 [-0.35650283 -0.5672024  -0.9969993  ... -0.966227   -0.37799835
  -0.03421372]
 [-0.83635956 -0.7908453  -0.9987985  ... -0.985      -0.7406638
   0.515569  ]
 [-0.31294173 -0.56006444 -0.99648726 ... -0.9616357  -0.3852686
  -0.04569644]]


In [33]:
input_vectors = vec_embeddings[:]
print(np.shape(input_vectors[:]))

(50, 768)


In [34]:
x_train = np.asarray(input_vectors)
print(x_train[:,:,None].shape)

(50, 768, 1)


In [35]:
output_vectors = np.array([i for i in range(50)])
print(np.shape(output_vectors))

(50,)


In [ ]:
#model.fit(x=x_train[:,:], y=output_vectors[:,None], epochs=1000)
model.fit(x=x_train[:,:,None], y=output_vectors[:,None], epochs=100)

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(1, 768, 1)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])
